In [1]:
import dkist

In [2]:
from astropy.modeling.fitting import TRFLSQFitter
import numpy as np
import astropy.modeling.models as m
import astropy.units as u

In [3]:
import matplotlib.pyplot as plt

In [4]:
from astropy.modeling.fitting import TRFLSQFitter
import numpy as np

In [5]:
asdf_file = "/home/tom/Data/DKIST/prod/pid_2_114/ALDLJ/VISP_L1_20231016T220247_ALDLJ.asdf"

In [6]:
visp = dkist.load_dataset(asdf_file)

In [7]:
wave = visp[0, :, 1000].axis_world_coords("em.wl")[0]

In [8]:
line_1 = m.Lorentz1D(amplitude=-0.6*u.ct, fwhm=0.1*u.nm, x_0=854.3*u.nm)
line_1_constrained = line_1.copy()
line_1_constrained.x_0.min = 854.25*u.nm
line_1_constrained.x_0.max = 854.35*u.nm

line_2 = m.Lorentz1D(amplitude=-0.25*u.ct, fwhm=0.01*u.nm, x_0=853.98*u.nm)
line_2_constrained = line_2.copy()
line_2_constrained.x_0.min = 853.95
line_2_constrained.x_0.max = 854.00

line_3 = m.Lorentz1D(amplitude=-0.15*u.ct, fwhm=0.01*u.nm, x_0=854.08*u.nm)
line_3_constrained = line_3.copy()
line_3_constrained.x_0.min = 854.05
line_3_constrained.x_0.max = 854.13

In [9]:
model_constrained = (
    m.Const1D(1*u.ct) + 
    line_1_constrained +
    line_2_constrained +
    line_3_constrained
)

model = (
    m.Const1D(1*u.ct) + 
    line_1 +
    line_2 +
    line_3
)

In this section we compare the performance of the fitting for a single chunk in our implementation of the parallel fitter, versus the fastest we could do in Python which is to write a hard-coded function with the model.

### Comparing model evaluation performance

In [10]:
def hardcoded_model(
    x,
    amplitude_0,
    amplitude_1,
    x_0_1,
    fwhm_1,
    amplitude_2,
    x_0_2,
    fwhm_2,
    amplitude_3,
    x_0_3,
    fwhm_3
):

    gamma_sq_1 = (fwhm_1 / 2.0) ** 2
    gamma_sq_2 = (fwhm_2 / 2.0) ** 2
    gamma_sq_3 = (fwhm_3 / 2.0) ** 2

    model = (
        amplitude_0
        + amplitude_1 * gamma_sq_1 / (gamma_sq_1 + (x - x_0_1) ** 2)
        + amplitude_2 * gamma_sq_2 / (gamma_sq_2 + (x - x_0_2) ** 2)
        + amplitude_3 * gamma_sq_3 / (gamma_sq_3 + (x - x_0_3) ** 2)
    )
    
    return model

In [11]:
model_constrained_nounit = model_constrained.without_units_for_data(x=1 * u.nm, y=1 * u.ct)

In [12]:
wave_nm = wave.to_value('nm')

In [13]:
%timeit model_constrained_nounit(wave_nm)

236 μs ± 851 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [14]:
x0 = [getattr(model_constrained_nounit, name).value for name in model_constrained_nounit.param_names]

In [15]:
%timeit hardcoded_model(wave_nm, *x0)

17 μs ± 30.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### Comparing fitting performance

In [16]:
from astropy.modeling.fitting_parallel import fit_models_to_chunk

In [17]:
chunk = np.array(visp[0].data[:,:100])
chunk_shape = chunk.shape
chunk_shape

(937, 100)

In [18]:
parameter_arrays = [np.broadcast_to(getattr(model_constrained_nounit, param).value,chunk_shape) for param in model_constrained_nounit.param_names]

In [19]:
%%time

fitted_parameters = fit_models_to_chunk(chunk,
                    *parameter_arrays,
                    model=model_constrained_nounit,
                    iterating_axes=(1,),
                    fitting_axes=(0,),
                    world=(wave,),
                    block_info=[0],
                    fitter=TRFLSQFitter(),
                   )

CPU times: user 12.5 s, sys: 5.03 ms, total: 12.5 s
Wall time: 4.17 s


In [20]:
def hardcoded_objective_function(
    params,
    x,
    y,
):    
    return  y - hardcoded_model(x, *params)

bounds = ([-np.inf, -np.inf, 854.25, -np.inf, -np.inf, 853.95, -np.inf, -np.inf, 854.05, -np.inf],
          [np.inf, np.inf, 854.35, np.inf, np.inf, 854.00, np.inf, np.inf, 854.13, np.inf])

from scipy.optimize import least_squares

def fit_models_to_chunk_hardcoded(data, *parameters):

    fitted_parameters = []
    
    for i in range(chunk.shape[1]):

        x0 = tuple(p[0, i] for p in parameters)
        
        result = least_squares(hardcoded_objective_function,
                               x0,
                               jac='2-point',
                               diff_step=np.float64(0.0001220703125),
                               bounds=bounds,
                               method='trf',
                               args=(wave.to_value('nm'),chunk[:, i]),
                               max_nfev=100,
                               xtol=1e-7)
        
        fitted_parameters.append(result.x)

    return np.array(fitted_parameters)

In [21]:
x0 = [getattr(model_constrained_nounit, name).value for name in model_constrained_nounit.param_names]

In [22]:
%%time

fitted_parameters_hardcoded = fit_models_to_chunk_hardcoded(chunk, *parameter_arrays)

CPU times: user 2.27 s, sys: 148 μs, total: 2.27 s
Wall time: 758 ms


In [23]:
np.testing.assert_allclose(fitted_parameters_hardcoded.T, fitted_parameters)